In [43]:
import pandas as pd
from pathlib import Path
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from datetime import timedelta
import re, unicodedata
import io

In [15]:
from pathlib import Path


PROJECT_ROOT = Path.cwd().parents[2]

DATA_RAW = PROJECT_ROOT / "data" / "raw" / "phishing"
DATA_INTERIM = PROJECT_ROOT / "data" / "interim" / "phishing"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed" / "phishing"

# Aseguramos carpetas
DATA_RAW.mkdir(parents=True, exist_ok=True)
DATA_INTERIM.mkdir(parents=True, exist_ok=True)
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

print("PROJECT_ROOT =", PROJECT_ROOT)
print("RAW       =", DATA_RAW)
print("INTERIM   =", DATA_INTERIM)
print("PROCESSED =", DATA_PROCESSED)


PROJECT_ROOT = /Users/test/Desktop/phishing-detector
RAW       = /Users/test/Desktop/phishing-detector/data/raw/phishing
INTERIM   = /Users/test/Desktop/phishing-detector/data/interim/phishing
PROCESSED = /Users/test/Desktop/phishing-detector/data/processed/phishing


In [17]:
# Nombre exacto del archivo que descargaste de PhishTank
fname = "phishtank_online_valid_20250728_120548.csv"


In [18]:
# Rutas de entrada y salida usando las carpetas definidas en Celda 0
INPUT = DATA_RAW / fname
OUTPUT_SNAPSHOT = DATA_INTERIM / "phishtank_loaded.parquet"


In [19]:
# Carga del CSV desde RAW
df = pd.read_csv(INPUT, low_memory=False)


In [20]:
# Guardamos snapshot de carga en formato Parquet
df.to_parquet(OUTPUT_SNAPSHOT, index=False)


In [21]:
# Información rápida de control
print(f"✅ Carga OK: {len(df):,} filas")
print("Snapshot →", OUTPUT_SNAPSHOT)
print("Primeras columnas:", list(df.columns)[:8])
df.head(3)


✅ Carga OK: 51,716 filas
Snapshot → /Users/test/Desktop/phishing-detector/data/interim/phishing/phishtank_loaded.parquet
Primeras columnas: ['phish_id', 'url', 'phish_detail_url', 'submission_time', 'verified', 'verification_time', 'online', 'target']


,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,9167131,https://teamvoice.m-pages.com/,http://www.phishtank.com/phish_detail.php?phis...,2025-07-28T08:55:52+00:00,yes,2025-07-28T09:02:54+00:00,yes,Other
1,9167130,https://merrimsg.m-pages.com/,http://www.phishtank.com/phish_detail.php?phis...,2025-07-28T08:55:29+00:00,yes,2025-07-28T09:02:54+00:00,yes,Other
2,9167129,https://tinkabee.m-pages.com/,http://www.phishtank.com/phish_detail.php?phis...,2025-07-28T08:55:01+00:00,yes,2025-07-28T09:02:54+00:00,yes,Other


In [23]:
df2 = df.copy()

In [24]:
# 1) Cabeceras consistentes
df2.columns = [c.strip().lower() for c in df2.columns]


In [25]:
# 2) Mapeo flexible de columnas clave a nombres canónicos
col_map = {}
for c in df2.columns:
    if c == "url":
        col_map[c] = "url"
    elif c in ["submission_time", "submitted_at", "date_submitted", "submissiondate"]:
        col_map[c] = "submission_time"
    elif c in ["verification_time", "verified_at", "verificationdate"]:
        col_map[c] = "verification_time"
    elif c in ["verified", "is_verified", "valid", "status_verified"]:
        col_map[c] = "verified"
    elif c in ["target", "brand", "entity", "company"]:
        col_map[c] = "target"

df2 = df2.rename(columns=col_map)


In [26]:
# 3) Parseo robusto de fechas si existen
for dc in ["submission_time", "verification_time"]:
    if dc in df2.columns:
        df2[dc] = pd.to_datetime(df2[dc], errors="coerce", utc=True)


In [27]:
presentes = [c for c in ["url", "submission_time", "verification_time", "verified", "target"] if c in df2.columns]
faltan = [c for c in ["url", "submission_time", "verification_time", "verified", "target"] if c not in df2.columns]


In [28]:
print("✅ Columnas presentes:", presentes)
print("⚠️  Columnas ausentes:", faltan)


✅ Columnas presentes: ['url', 'submission_time', 'verification_time', 'verified', 'target']
⚠️  Columnas ausentes: []


In [31]:
# 1) Elegimos la columna de fecha preferente
#    - Usamos submission_time si existe; si no, verification_time.
fecha_col = "submission_time" if "submission_time" in df2.columns else "verification_time"


In [32]:
# 2) Fecha de corte: último año.
#    df2 está en UTC (parseado así), por coherencia calculamos ahora en UTC.
hoy_utc = pd.Timestamp.now(tz="UTC")
corte = hoy_utc - timedelta(days=365)


In [33]:
# 3) Normalizamos 'verified' a booleano (por si viene como 'True', '1', 'yes', etc.)
if "verified" in df2.columns:
    def to_bool(v):
        if pd.isna(v): 
            return False
        if isinstance(v, (int, float)): 
            return v == 1
        s = str(v).strip().lower()
        return s in ("true", "1", "yes", "y", "verified")
    df2["verified_bool"] = df2["verified"].apply(to_bool)
else:
    df2["verified_bool"] = False  # no debería pasar porque verified está presente, pero dejamos el fallback


In [34]:
# 4) Aplicamos los filtros
df3 = df2[(df2[fecha_col] >= corte) & (df2["verified_bool"] == True)].copy()


In [35]:
# 5) Controles y métricas rápidas
total_inicial = len(df2)
total_ultimo_anio = len(df2[df2[fecha_col] >= corte])
total_final = len(df3)


In [36]:
print("=== FILTRO PHISHTANK ===")
print(f"Fecha de corte (último año desde): {corte}")
print(f"Columna de fecha utilizada: {fecha_col}")
print(f"- Total inicial: {total_inicial:,}")
print(f"- Del último año: {total_ultimo_anio:,}")
print(f"- Último año + verified: {total_final:,}")


=== FILTRO PHISHTANK ===
Fecha de corte (último año desde): 2024-08-13 13:59:48.682582+00:00
Columna de fecha utilizada: submission_time
- Total inicial: 51,716
- Del último año: 27,926
- Último año + verified: 27,926


In [38]:
# Ruta a tu tabla (ajusta si está en otra carpeta)
COMPANY_FILE = PROJECT_ROOT / "docs" / "tabla_empresas.md"


In [45]:

# 0) Localiza el archivo (ajusta si lo tienes en otra ruta)
COMPANY_FILE = (PROJECT_ROOT / "docs" / "tabla_empresas.md") if 'PROJECT_ROOT' in globals() else Path("docs/tabla_empresas.md")
if not COMPANY_FILE.exists():
    # Fallback si estás trabajando aquí y lo subiste a /mnt/data
    alt = Path("/mnt/data/tabla_empresas.md")
    if alt.exists():
        COMPANY_FILE = alt
    else:
        raise FileNotFoundError(f"No encuentro tabla_empresas.md en {COMPANY_FILE} ni en /mnt/data/")

# --- Utils ---
def strip_accents(s: str) -> str:
    if not isinstance(s, str): return ""
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def norm(s: str) -> str:
    return strip_accents(str(s)).strip().lower()

def is_separator(line: str) -> bool:
    s = line.strip()
    return s and set(s) <= set("|:- ")

def split_md_row(line: str) -> list:
    line = line.strip()
    if line.startswith("|"): line = line[1:]
    if line.endswith("|"): line = line[:-1]
    return [c.strip() for c in line.split("|")]

# 1) Lee el .md y detecta qué columna es "Empresa" (case-insensitive, sin tildes)
lines = COMPANY_FILE.read_text(encoding="utf-8").splitlines()
header = None
emp_idx = None
data_started = False
empresas_raw = []

for ln in lines:
    if not ln.strip(): 
        continue
    if is_separator(ln):
        continue
    if "|" not in ln:
        continue

    cols = split_md_row(ln)

    if header is None:
        # Primera fila con '|' que no sea separador: la tratamos como header
        header = cols
        # Busca índice de "Empresa" de forma robusta
        header_norm = [norm(h) for h in header]
        try:
            emp_idx = header_norm.index("empresa")
        except ValueError:
            # Si no hay columna 'Empresa', por simplicidad usamos la 2ª columna (índice 1) si existe
            emp_idx = 1 if len(header) > 1 else 0
        continue

    # Filas de datos
    # Normaliza longitud al nº de columnas del header por seguridad
    if len(cols) < len(header):
        cols += [""] * (len(header) - len(cols))
    # Si se pasan, compacta extras en la última celda
    elif len(cols) > len(header):
        cols = cols[:len(header)-1] + [" | ".join(cols[len(header)-1:])]

    nombre = cols[emp_idx].strip()
    if nombre and nombre.lower() != "empresa":
        empresas_raw.append(nombre)

# 2) Limpieza mínima de nombres y construcción del set
#    - normaliza tildes/minúsculas
#    - elimina duplicados
empresas_limpias = sorted({e.strip() for e in empresas_raw if e.strip()})
set_empresas = {norm(e) for e in empresas_limpias if e.strip()}

print(f"✅ Empresas extraídas: {len(empresas_limpias)} (únicas normalizadas: {len(set_empresas)})")
print("Ejemplos:", empresas_limpias[:10])


✅ Empresas extraídas: 104 (únicas normalizadas: 104)
Ejemplos: ['Abanca', 'Adidas', 'Adobe', 'Agencia Tributaria (AEAT)', 'Airbnb', 'AliExpress', 'Amazon', 'Amazon Logistics', 'American Express', 'Apple (Apple ID)']


In [47]:
# === 3B: Construir BRAND_TOKENS desde 'empresas_limpias' / 'set_empresas' ===


# Partimos de 'empresas_limpias' que acabas de extraer del .md (si no, cámbialo por tu lista)
assert 'empresas_limpias' in globals() and len(empresas_limpias) > 0, "No encuentro 'empresas_limpias'. Ejecuta antes la celda de extracción del .md."

def strip_accents(s: str) -> str:
    if not isinstance(s, str): return ""
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def norm(s: str) -> str:
    return strip_accents(str(s)).lower().strip()

def split_name_variants(name: str):
    """
    Divide 'Agencia Tributaria (AEAT)' -> ['Agencia Tributaria', 'AEAT']
    'Apple (Apple ID)' -> ['Apple', 'Apple ID']
    """
    name = name.strip()
    parts = []
    m = re.match(r'^(.*?)(?:\((.*?)\))?$', name)
    if m:
        main = m.group(1).strip()
        alt  = (m.group(2) or "").strip()
        if main: parts.append(main)
        if alt:  parts.append(alt)
    else:
        parts.append(name)
    return [p for p in parts if p]

def tokenize_brand(name: str):
    """
    Normaliza y tokeniza a unidades útiles:
    - 'Agencia Tributaria' -> ['agencia','tributaria','agencia-tributaria']
    - 'American Express'   -> ['american','express','american-express','amex']
    - 'AEAT' -> ['aeat']
    """
    s = norm(name)
    # tokens alfanuméricos >=3
    toks = [t for t in re.split(r'[^a-z0-9]+', s) if len(t) >= 3]
    bag = set(toks)
    # combina tokens contiguos con '-' (bigramas) para capturar marcas compuestas
    for i in range(len(toks)-1):
        bag.add(f"{toks[i]}-{toks[i+1]}")
    # sinónimos puntuales
    if "american" in bag and "express" in bag:
        bag.add("amex")
    if "agencia" in bag and "tributaria" in bag:
        bag.add("aeat")
    if "apple" in bag and ("id" in s or "apple id" in s):
        bag.add("apple-id")
    if "la" in bag and "caixa" in bag:
        bag.add("lacaixa"); bag.add("la-caixa"); bag.add("caixa")
    if "caixabank" in s:
        bag.add("caixa"); bag.add("imagin"); bag.add("imaginbank")
    if "bbva" in bag:
        bag.add("bbva-espana"); bag.add("bbvaespana")
    if "masmovil" in bag:
        bag.add("mas-movil")
    if "seguridad" in bag and "social" in bag:
        bag.add("seguridad-social")
    return bag

# Construir el set final de tokens de marca
BRAND_TOKENS = set()
for name in empresas_limpias:
    for variant in split_name_variants(name):
        BRAND_TOKENS |= tokenize_brand(variant)

# Quitar términos demasiado genéricos
BANLIST = {"banco","banca","fintech","cloud","correo","redes","publico","público",
           "energia","telecomunicaciones","ecommerce","retail","streaming",
           "turismo","viajes","pagos","pago","online","plataforma","empresa","servicios","logistics"}
BRAND_TOKENS = {t for t in BRAND_TOKENS if t not in BANLIST and len(t) >= 3}

print(f"✅ BRAND_TOKENS listo: {len(BRAND_TOKENS)} tokens")
print(sorted(list(BRAND_TOKENS))[:40])


✅ BRAND_TOKENS listo: 144 tokens
['365', 'abanca', 'adidas', 'adobe', 'aeat', 'agencia', 'agencia-tributaria', 'airbnb', 'aliexpress', 'amazon', 'amazon-logistics', 'american', 'american-express', 'amex', 'apple', 'apple-id', 'banca-march', 'banco-espana', 'banco-santander', 'bankinter', 'binance', 'bit2me', 'bizum', 'booking', 'booking-com', 'caixa', 'caixabank', 'cajamar', 'carrefour', 'catastro', 'cepsa', 'coinbase', 'com', 'consumer', 'consumer-finance', 'correos', 'corte', 'corte-ingles', 'decathlon', 'dgt']


In [49]:
import re

# 1) Elimina tokens solo numéricos y sin letras (ej: "365"), y TLDs/comunes
DROP_EXACT = {
    "com","www","login","secure","account","support","help","portal","webmail","mail","online",
    "consumer","logistics"  # monos muy amplios
}
BRAND_TOKENS = {
    t for t in BRAND_TOKENS
    if re.search("[a-z]", t) and not t.isdigit() and t not in DROP_EXACT
}

# 2) Prefiere compuestos antes que monos ambiguos si ya existe el bigrama correspondiente
#    (si tienes "corte-ingles", quitamos "corte" e "ingles"; si tienes "american-express", quitamos "american"/"express"; etc.)
MONOS_AMBIGUOS = {"corte","ingles","american","express","agencia","tributaria"}
bigrams = {t for t in BRAND_TOKENS if "-" in t}
parts_from_bigrams = set()
for bg in bigrams:
    parts_from_bigrams |= set(bg.split("-"))

BRAND_TOKENS = {
    t for t in BRAND_TOKENS
    if not (t in MONOS_AMBIGUOS and t in parts_from_bigrams)
}

print(f"✅ BRAND_TOKENS depurado: {len(BRAND_TOKENS)}")
print(sorted(list(BRAND_TOKENS))[:40])


✅ BRAND_TOKENS depurado: 135
['abanca', 'adidas', 'adobe', 'aeat', 'agencia-tributaria', 'airbnb', 'aliexpress', 'amazon', 'amazon-logistics', 'american-express', 'amex', 'apple', 'apple-id', 'banca-march', 'banco-espana', 'banco-santander', 'bankinter', 'binance', 'bit2me', 'bizum', 'booking', 'booking-com', 'caixa', 'caixabank', 'cajamar', 'carrefour', 'catastro', 'cepsa', 'coinbase', 'consumer-finance', 'correos', 'corte-ingles', 'decathlon', 'dgt', 'dhl', 'digi', 'disney', 'docusign', 'drive', 'dropbox']


In [50]:
# === 3C-bis: Re-scoring ES (no excluyente) con BRAND_TOKENS depurados ===
import re
import unicodedata
from urllib.parse import urlparse
import pandas as pd

# 0) Dataset base
if 'df3' in globals():
    base = df3.copy()
elif 'df2' in globals():
    base = df2.copy()
elif 'df' in globals():
    base = df.copy()
else:
    raise RuntimeError("No encuentro df/df2/df3. Ejecuta antes las celdas de carga y estandarización.")

# 1) Utils
def norm(s: str) -> str:
    if not isinstance(s, str): return ""
    s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
    return s.lower().strip()

def host(url: str) -> str:
    try: return urlparse(url).netloc.lower()
    except: return ""

def route(url: str) -> str:
    try:
        u = urlparse(url)
        r = (u.path or "") + ("?" + u.query if u.query else "")
        return norm(r)
    except:
        return ""

def has_token_with_boundaries(s: str, token: str) -> bool:
    # separadores comunes en URLs: / . _ -
    pattern = rf'(^|[\/\._\-]){re.escape(token)}([\/\._\-]|$)'
    return re.search(pattern, s) is not None

# 2) Preparación columnas normalizadas
df_sc = base.copy()
df_sc["domain"] = df_sc["url"].apply(host)
df_sc["route"]  = df_sc["url"].apply(route)
df_sc["domain_norm"] = df_sc["domain"].apply(norm)
df_sc["target_norm"] = df_sc["target"].apply(norm) if "target" in df_sc.columns else ""

# 3) Señales
# T1 (3 pts): el target menciona alguna marca de tu lista
df_sc["f_target_in_list"] = df_sc["target_norm"].apply(lambda t: any(tok in t for tok in BRAND_TOKENS))

# U1 (2 pts): token de marca en dominio o ruta con bordes
df_sc["f_brand_in_url"] = df_sc.apply(
    lambda r: any(has_token_with_boundaries(r["domain_norm"], tok) or
                  has_token_with_boundaries(r["route"], tok) for tok in BRAND_TOKENS),
    axis=1
)

# U2 (2 pts): pistas españolas en la ruta
SPANISH_HINTS = [
    "iniciar-sesion","iniciarsesion","acceso","entrar","identificacion","verificar","verificacion",
    "actualizar","actualizacion","confirmar","confirmacion","seguridad","seguro",
    "recuperar","recuperacion","restablecer","contrasena","contrasenna","clave",
    "cuenta","banco","iban","iban-es","bizum","transferencia","tarjeta","movimientos",
    "aeat","agencia-tributaria","hacienda","dgt","seguridad-social","seguridad social","sepe","catastro",
    "certificado","cl@ve","cl@ve-permanente","cl@ve permanente","firma","factura","facturacion"
]
pat_hints = re.compile("|".join(map(re.escape, SPANISH_HINTS)))
df_sc["f_spanish_hints"] = df_sc["route"].apply(lambda s: bool(pat_hints.search(s)))

# U3 (1 pt): locale ES o símbolo €
SPANISH_LOCALE = ["lang=es","locale=es","locale=es-es","/es/"]
df_sc["f_locale_es"]  = df_sc["route"].apply(lambda s: any(h in s for h in SPANISH_LOCALE))
df_sc["f_euro_symbol"] = df_sc["route"].str.contains("%e2%82%ac|€", regex=True)

# U4 (1 pt): prefijo +34
df_sc["f_plus34"] = df_sc["route"].str.contains(r"\+34")

# D1 (1 pt): TLD .es (suma, NO excluye)
df_sc["f_es_tld"] = df_sc["domain_norm"].str.endswith(".es")

# 4) Score y decisión
THRESH = 3
score = (df_sc["f_target_in_list"] * 3
        + df_sc["f_brand_in_url"] * 2
        + df_sc["f_spanish_hints"] * 2
        + df_sc["f_locale_es"] * 1
        + df_sc["f_euro_symbol"] * 1
        + df_sc["f_plus34"] * 1
        + df_sc["f_es_tld"] * 1)

df_sc["es_score"] = score
candidatas = df_sc[df_sc["es_score"] >= THRESH].copy()

print("=== RE-SCORING (BRAND_TOKENS depurados) ===")
print(f"Base: {len(df_sc):,} | Candidatas ES: {len(candidatas):,} ({len(candidatas)/max(len(df_sc),1):.1%}) | Umbral: {THRESH}")

# 5) Guardar en INTERIM para auditoría
out_es = DATA_INTERIM / "phishtank_es_candidates_scored.csv"
cols_out = [c for c in ["url","submission_time","verification_time","verified","target","domain",
                        "es_score","f_target_in_list","f_brand_in_url","f_spanish_hints",
                        "f_locale_es","f_euro_symbol","f_plus34","f_es_tld"] if c in candidatas.columns]
candidatas[cols_out].to_csv(out_es, index=False)
print("💾 Guardado (INTERIM) →", out_es)

# Vista de muestra
display(candidatas.head(10))


=== RE-SCORING (BRAND_TOKENS depurados) ===
Base: 27,926 | Candidatas ES: 535 (1.9%) | Umbral: 3
💾 Guardado (INTERIM) → /Users/test/Desktop/phishing-detector/data/interim/phishing/phishtank_es_candidates_scored.csv


,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target,verified_bool,domain,...,domain_norm,target_norm,f_target_in_list,f_brand_in_url,f_spanish_hints,f_locale_es,f_euro_symbol,f_plus34,f_es_tld,es_score
31,9167080,https://translators-exec.vercel.app/facebook-m...,http://www.phishtank.com/phish_detail.php?phis...,2025-07-28 07:51:39+00:00,yes,2025-07-28 08:03:01+00:00,yes,Facebook,True,translators-exec.vercel.app,...,translators-exec.vercel.app,facebook,True,True,False,False,False,False,False,5
32,9167079,https://translators-exec.vercel.app/wrapper-float,http://www.phishtank.com/phish_detail.php?phis...,2025-07-28 07:51:17+00:00,yes,2025-07-28 08:03:01+00:00,yes,Facebook,True,translators-exec.vercel.app,...,translators-exec.vercel.app,facebook,True,False,False,False,False,False,False,3
33,9167078,https://rebrand.ly/wc6756,http://www.phishtank.com/phish_detail.php?phis...,2025-07-28 07:50:11+00:00,yes,2025-07-28 08:03:01+00:00,yes,Facebook,True,rebrand.ly,...,rebrand.ly,facebook,True,False,False,False,False,False,False,3
63,9167007,https://help-sso-coinbasehelpe.webflow.io/,http://www.phishtank.com/phish_detail.php?phis...,2025-07-28 04:49:04+00:00,yes,2025-07-28 04:56:14+00:00,yes,Coinbase,True,help-sso-coinbasehelpe.webflow.io,...,help-sso-coinbasehelpe.webflow.io,coinbase,True,False,False,False,False,False,False,3
97,9166963,https://sustainableunityforupliftedandrespecte...,http://www.phishtank.com/phish_detail.php?phis...,2025-07-28 03:33:15+00:00,yes,2025-07-28 03:41:41+00:00,yes,Australian Taxation Office,True,sustainableunityforupliftedandrespectedcitizen...,...,sustainableunityforupliftedandrespectedcitizen...,australian taxation office,True,False,False,False,False,False,False,3
543,9166260,https://netflix-pagos.blog/,http://www.phishtank.com/phish_detail.php?phis...,2025-07-26 21:16:53+00:00,yes,2025-07-26 21:21:54+00:00,yes,Netflix,True,netflix-pagos.blog,...,netflix-pagos.blog,netflix,True,True,False,False,False,False,False,5
544,9166257,https://facebook-verifyidentity.com/,http://www.phishtank.com/phish_detail.php?phis...,2025-07-26 21:11:11+00:00,yes,2025-07-26 21:21:54+00:00,yes,Facebook,True,facebook-verifyidentity.com,...,facebook-verifyidentity.com,facebook,True,True,False,False,False,False,False,5
545,9166258,https://www.facebook-verifyidentity.com/,http://www.phishtank.com/phish_detail.php?phis...,2025-07-26 21:11:11+00:00,yes,2025-07-26 21:21:54+00:00,yes,Facebook,True,www.facebook-verifyidentity.com,...,www.facebook-verifyidentity.com,facebook,True,True,False,False,False,False,False,5
549,9166252,https://cooinjbasepriologiinj.godaddysites.com/,http://www.phishtank.com/phish_detail.php?phis...,2025-07-26 21:02:02+00:00,yes,2025-07-26 21:12:32+00:00,yes,Coinbase,True,cooinjbasepriologiinj.godaddysites.com,...,cooinjbasepriologiinj.godaddysites.com,coinbase,True,False,False,False,False,False,False,3
578,9166143,https://net-renewflix-pt.com/,http://www.phishtank.com/phish_detail.php?phis...,2025-07-26 13:00:04+00:00,yes,2025-07-26 13:03:01+00:00,yes,Netflix,True,net-renewflix-pt.com,...,net-renewflix-pt.com,netflix,True,False,False,False,False,False,False,3


In [51]:
import re
from urllib.parse import urlparse, parse_qsl, urlunparse, urlencode

# Trabajamos sobre una copia para no tocar el original
dfn = candidatas.copy()

# ----------------------
# 1) Normalizar esquema y dominio (https y dominio en minúsculas)
# ----------------------
def normalize_scheme_and_host(url: str) -> str:
    try:
        u = urlparse(url.strip())
        scheme = "https" if u.scheme in ("http", "https", "") else u.scheme
        netloc = u.netloc.lower()
        return urlunparse((scheme, netloc, u.path, u.params, u.query, u.fragment))
    except:
        return url

dfn["url"] = dfn["url"].astype(str).apply(normalize_scheme_and_host)

# ----------------------
# 2) Quitar "/" final (evita contar /login y /login/ como distintos)
# ----------------------
dfn["url"] = dfn["url"].str.rstrip("/")

# ----------------------
# 3) Eliminar parámetros de tracking (no cambian el contenido)
# ----------------------
DROP_PARAMS = {
    # marketing / analítica
    "utm_source","utm_medium","utm_campaign","utm_term","utm_content","gclid","fbclid",
    # sesiones / tracking varios
    "sessionid","sid","phpsessid","jsessionid","trackid",
    # refts
    "ref","refid","clickid"
}
def drop_tracking_params(url: str) -> str:
    try:
        u = urlparse(url)
        # filtra el querystring
        q = [(k, v) for (k, v) in parse_qsl(u.query, keep_blank_values=True) if k.lower() not in DROP_PARAMS]
        new_q = urlencode(q, doseq=True)
        return urlunparse((u.scheme, u.netloc, u.path, u.params, new_q, u.fragment)).rstrip("?")
    except:
        return url

dfn["url"] = dfn["url"].apply(drop_tracking_params)

# ----------------------
# 4) Deduplicado (primero por URL exacta, luego por dominio+path)
# ----------------------
before = len(dfn)
dfn = dfn.drop_duplicates(subset=["url"]).copy()
after_url = len(dfn)

# Extrae dominio y path para el dedupe por ruta
dfn["domain"] = dfn["url"].apply(lambda u: urlparse(u).netloc)
dfn["path"]   = dfn["url"].apply(lambda u: urlparse(u).path)

dfn = dfn.drop_duplicates(subset=["domain", "path"]).copy()
after_path = len(dfn)

print("=== NORMALIZACIÓN & DEDUP ===")
print(f"Antes: {before:,} | Únicas por URL: {after_url:,} | Únicas por dominio+path: {after_path:,}")

# ----------------------
# 5) Guardados (dejamos en INTERIM para revisar; cuando valides, pasamos a PROCESSED)
# ----------------------
norm_path = DATA_INTERIM / "phishtank_es_normalized.parquet"
dfn.to_parquet(norm_path, index=False)
print("💾 INTERIM →", norm_path)

# Si quieres ya el CSV “final”, descomenta estas dos líneas:
# final_csv = DATA_PROCESSED / "phishtank_es_clean.csv"
# dfn.to_csv(final_csv, index=False)  # ← déjalo para cuando confirmes la revisión manual

# ----------------------
# 6) Inspecciones rápidas (te ayudan a entender lo que quedó)
# ----------------------
# Top 10 dominios (para detectar campañas repetidas)
print("\nTop dominios (muestra):")
display(dfn["domain"].value_counts().head(10))

# Top 10 targets (qué marcas salen más)
if "target" in dfn.columns:
    print("\nTop targets (muestra):")
    display(dfn["target"].value_counts().head(10))

# Muestra explicativa (con es_score y banderas)
cols_exp = [c for c in ["url","target","es_score",
                        "f_target_in_list","f_brand_in_url","f_spanish_hints",
                        "f_locale_es","f_euro_symbol","f_plus34","f_es_tld"] if c in dfn.columns]
print("\nMuestra explicativa:")
display(dfn[cols_exp].head(10))


=== NORMALIZACIÓN & DEDUP ===
Antes: 535 | Únicas por URL: 525 | Únicas por dominio+path: 520
💾 INTERIM → /Users/test/Desktop/phishing-detector/data/interim/phishing/phishtank_es_normalized.parquet

Top dominios (muestra):


domain
qrco.de                   11
docs.google.com            9
l.ead.me                   8
t.co                       6
sites.google.com           5
rebrand.ly                 5
ln.run                     4
dmaraket.cc                3
forms.visme.co             3
secure3-primevideo.com     3
Name: count, dtype: int64


Top targets (muestra):


target
Netflix                                85
Coinbase                               65
Facebook                               64
Microsoft                              45
Sumitomo Mitsui Banking Corporation    36
Steam                                  34
Amazon.com                             27
Apple                                  24
eBay, Inc.                             21
Other                                  19
Name: count, dtype: int64


Muestra explicativa:


,url,target,es_score,f_target_in_list,f_brand_in_url,f_spanish_hints,f_locale_es,f_euro_symbol,f_plus34,f_es_tld
31,https://translators-exec.vercel.app/facebook-m...,Facebook,5,True,True,False,False,False,False,False
32,https://translators-exec.vercel.app/wrapper-float,Facebook,3,True,False,False,False,False,False,False
33,https://rebrand.ly/wc6756,Facebook,3,True,False,False,False,False,False,False
63,https://help-sso-coinbasehelpe.webflow.io,Coinbase,3,True,False,False,False,False,False,False
97,https://sustainableunityforupliftedandrespecte...,Australian Taxation Office,3,True,False,False,False,False,False,False
543,https://netflix-pagos.blog,Netflix,5,True,True,False,False,False,False,False
544,https://facebook-verifyidentity.com,Facebook,5,True,True,False,False,False,False,False
545,https://www.facebook-verifyidentity.com,Facebook,5,True,True,False,False,False,False,False
549,https://cooinjbasepriologiinj.godaddysites.com,Coinbase,3,True,False,False,False,False,False,False
578,https://net-renewflix-pt.com,Netflix,3,True,False,False,False,False,False,False


In [52]:
# === Depuración extra: excluir genéricos + reforzar marca + subir umbral ===
import re
from urllib.parse import urlparse
import pandas as pd

# 0) Cargar base: usa 'dfn' si existe; si no, lee el último normalizado
if 'dfn' in globals():
    base = dfn.copy()
else:
    base = pd.read_parquet(DATA_INTERIM / "phishtank_es_normalized.parquet")

# Asegura columnas 'domain' y 'path'
if "domain" not in base.columns or "path" not in base.columns:
    base["domain"] = base["url"].apply(lambda u: urlparse(str(u)).netloc)
    base["path"]   = base["url"].apply(lambda u: urlparse(str(u)).path)

# 1) Dominios genéricos a EXCLUIR (acortadores/hostings comunes)
GENERIC_SUFFIXES = {
    "docs.google.com", "sites.google.com", "drive.google.com", "forms.gle",
    "t.co", "bit.ly", "rebrand.ly", "ln.run", "l.ead.me", "qrco.de",
    "vercel.app", "godaddysites.com", "webflow.io", "blogspot.com", "wordpress.com",
    "forms.visme.co", "github.io", "glitch.me", "firebaseapp.com", "000webhostapp.com"
}
def is_generic(host: str) -> bool:
    d = (host or "").lower()
    return any(d == g or d.endswith("." + g) for g in GENERIC_SUFFIXES)

# 2) Preparación para chequear marca en target/URL (si faltan banderas, las creamos)
def norm(s: str) -> str:
    import unicodedata
    if not isinstance(s, str): return ""
    s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
    return s.lower().strip()

def has_token_with_boundaries(s: str, token: str) -> bool:
    # bordes comunes en URLs: / . _ -
    pat = rf'(^|[\/\._\-]){re.escape(token)}([\/\._\-]|$)'
    return re.search(pat, s) is not None

# target_norm / route_norm
if "target_norm" not in base.columns:
    base["target_norm"] = base["target"].apply(norm) if "target" in base.columns else ""
if "route" not in base.columns:
    base["route"] = base["url"].apply(lambda u: (urlparse(str(u)).path or "") + ("?" + urlparse(str(u)).query if urlparse(str(u)).query else ""))
base["route_norm"] = base["route"].apply(norm)
base["domain_norm"] = base["domain"].apply(norm)

# Señales de marca (si no existen)
if "f_target_in_list" not in base.columns:
    assert 'BRAND_TOKENS' in globals() and len(BRAND_TOKENS) > 0, "Necesito BRAND_TOKENS (ejecuta antes 3B)."
    base["f_target_in_list"] = base["target_norm"].apply(lambda t: any(tok in t for tok in BRAND_TOKENS))
if "f_brand_in_url" not in base.columns:
    base["f_brand_in_url"] = base.apply(
        lambda r: any(has_token_with_boundaries(r["domain_norm"], tok) or
                      has_token_with_boundaries(r["route_norm"], tok)
                      for tok in BRAND_TOKENS),
        axis=1
    )

# 3) Filtros: no genéricos + marca presente (target o URL) + umbral más estricto
THRESH_STRICT = 4  # si queda corto/estricto, ajusta a 3 o 5
mask = (
    (~base["domain"].apply(is_generic)) &
    ( base["f_target_in_list"] | base["f_brand_in_url"] ) &
    ( base["es_score"] >= THRESH_STRICT )
)
clean = base[mask].copy()

# 4) Métricas y guardado
print("=== DEPURACIÓN EXTRA ===")
print(f"Entrada: {len(base):,}")
print(f"- Excluye genéricos: {sum(base['domain'].apply(is_generic)):,}")
print(f"- Mantiene marca (target/url): {sum( (base['f_target_in_list']|base['f_brand_in_url']) ) :,}")
print(f"- es_score ≥ {THRESH_STRICT}: {sum(base['es_score'] >= THRESH_STRICT):,}")
print(f"→ SALIDA final: {len(clean):,} ({len(clean)/max(len(base),1):.1%})")

# Guardar
clean_path = DATA_PROCESSED / "phishtank_es_clean.csv"
clean.to_csv(clean_path, index=False)
print("💾 PROCESSED →", clean_path)

# Vistazo
print("\nTop dominios (clean):")
display(clean["domain"].value_counts().head(10))

print("\nMuestra explicativa:")
cols_exp = [c for c in ["url","target","es_score","f_target_in_list","f_brand_in_url"] if c in clean.columns]
display(clean[cols_exp].head(10))


=== DEPURACIÓN EXTRA ===
Entrada: 520
- Excluye genéricos: 165
- Mantiene marca (target/url): 520
- es_score ≥ 4: 91
→ SALIDA final: 61 (11.7%)
💾 PROCESSED → /Users/test/Desktop/phishing-detector/data/processed/phishing/phishtank_es_clean.csv

Top dominios (clean):


domain
b9xja.dgnsvwrk.es                        2
orange-re-regularisation.com             2
extremas.com.ar                          2
netflix-seguro.com                       2
support-netflix-signup.com               2
www.dropbox.com                          2
help-netflix-signup.com                  2
s102.servername.online                   1
172635-coinbase.com                      1
q6oiqoze74oj-pages-dev.translate.goog    1
Name: count, dtype: int64


Muestra explicativa:


,url,target,es_score,f_target_in_list,f_brand_in_url
543,https://netflix-pagos.blog,Netflix,5,True,True
544,https://facebook-verifyidentity.com,Facebook,5,True,True
545,https://www.facebook-verifyidentity.com,Facebook,5,True,True
604,https://help-netflix-signup.com,Netflix,5,True,True
605,https://help-netflix-signup.com/index.php,Netflix,5,True,True
1147,https://www.172635-coinbase.com,Coinbase,5,True,True
1148,https://172635-coinbase.com,Coinbase,5,True,True
2119,https://s102.servername.online/~codes783/wp-co...,ING Direct,5,True,True
2199,https://orange-re-regularisation.com,Orange,5,True,True
2200,https://orange-re-regularisation.com/pages/ind...,Orange,5,True,True
